<a href="https://colab.research.google.com/github/andreahorbach/SeminarMSV/blob/main/TR_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sitzung 3: Authorship Attribution und Machine Learning
Der Code zur heutigen Sitzung folgt teilweise Kapitel 5 und 6 aus "Getting Started with Natural Language Processing" von Ekaterina Kochmar.

**Schritt 1: Daten**
Wie letzte Woche benutzen wir das nltk Gutenberg Corpus. Wir wollen einen Klassifikator bauen, der lernt Texte von William Shakespeare von Texten von Jane Austen zu unterscheiden. Wir laden also wieder die entsprechenden Corpora.

In [ ]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from nltk.corpus import gutenberg

gutenberg.fileids()

In [ ]:
austen1 = gutenberg.sents('austen-emma.txt')
austen2 = gutenberg.sents('austen-persuasion.txt')
austen3 = gutenberg.sents('austen-sense.txt')
shakespeare1 = gutenberg.sents("shakespeare-caesar.txt")
shakespeare2 = gutenberg.sents("shakespeare-hamlet.txt")
shakespeare3 = gutenberg.sents("shakespeare-macbeth.txt")
print (austen1, len(austen1))
print (austen2, len(austen2))
print (austen3, len(austen3))
print (shakespeare1, len(shakespeare1))
print (shakespeare2, len(shakespeare2))
print (shakespeare3, len(shakespeare3))

**Schritt 2: Bücher in kleinere Abschnitte zerlegen**
beim klassischen Machine Learning brauchen wir möglichst viele Trainingsdaten. Daher zerlegen wir jedes Buch in Sätze. Zusätzlich bekommt jeder Satz ein Label, den Autorennamen.
Wir wollen verhindern, dass unser Klassifikator ein Topic lernt (Wenn der Text 'Hamlet' enthält, ist es von Shakespeare.) Deswegen kombinieren wir Trainingsdaten aus jeweils zwei Büchern pro Autor und testen auf dem dritten Buch.

In [ ]:
train_set = [(sent, "austen") for sent in austen1]
train_set += [(sent, "austen") for sent in austen2]
train_set += [(sent, "shakespeare") for sent in shakespeare1]
train_set += [(sent, "shakespeare") for sent in shakespeare2]
test_set = [(sent, "austen") for sent in austen3]
test_set += [(sent, "shakespeare") for sent in shakespeare3]
print (f"Training dataset size = {str(len(train_set))} sentences")
print (f"Test dataset size = {str(len(test_set))} sentences")

**Schritt 3: Ein erster Classifier**
Wie letzte Woche verwenden wir Längenmerkmale. Wie viele Wörter hat ein Satz? Wie viele Buchstaben hat ein Wort? Diese Merkmale extrahieren wir für jeden einzelnen Textabschnitt.

In [8]:
#average word length in characters
def avg_number_chars(text):
    total_chars = 0.0
    for word in text:
        total_chars += len(word)
    return float(total_chars)/float(len(text))

#number of wors in a text
def number_words(text):
    return float(len(text)) # len() gibt die Länge einer Liste an, in unserem Fall ist ein Satz eine Liste von WÖrtern

print(avg_number_chars(["Not", "so", "happy", ",", "yet", "much", "happyer"]))
print(number_words(["Not", "so", "happy", ",", "yet", "much", "happyer"]))

3.5714285714285716
7.0


**Aufgabe 1**
Erweitern Sie den Feature-Extractions-Code oben, so dass er zusätzlich auch noch das Verhältnis von unterschiedlichen Wörtern zur Gesamtzahl der Wörter (Type-Token-Ratio) berücksichtigt. Tipp: Bedienen Sie sich im Code von letzter Woche.

In [ ]:
def initialize_dataset(source):
    all_features = [] # Merkmale
    targets = [] # Gold labels
    for (sent, label) in source:
        feature_list=[]
        feature_list.append(avg_number_chars(sent))
        feature_list.append(number_words(sent))
        all_features.append(feature_list)
        if label=="austen": targets.append(0)
        else: targets.append(1)
    return all_features, targets

train_data, train_targets = initialize_dataset(train_set)
test_data, test_targets = initialize_dataset(test_set)

print (len(train_data), len(train_targets))
print (len(test_data), len(test_targets))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn import metrics

text_clf = DecisionTreeClassifier(random_state=42)
text_clf.fit(train_data, train_targets)

def evaluate(predicted, targets):
    print(np.mean(predicted == targets)) # wie häufig ist das predictete Label das gleiche wie das gold label? Das ist die Accuracy.
    print(metrics.confusion_matrix(targets, predicted))
    print(metrics.classification_report(targets, predicted))

# we test and evaluate on the test data
predicted = text_clf.predict(test_data)
evaluate(predicted, test_targets)

# for comparison, we also apply and evaluate on the training data
predicted = text_clf.predict(train_data)
evaluate(predicted, train_targets)



**Aufgabe 2** Vergleichen Sie die Performanz auf den Trainingsdaten und den Testdaten. War das Ergebnis so erwartbar?

**Schritt 4:** Wir nutzen auch Häufigkeiten der einzelnen Wortarten als Features. (Sie sehen, dass wir viel Code von letzter Woche wiederverwenden.)

In [ ]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from collections import Counter

nltk.download('averaged_perceptron_tagger')

def map_fine_to_coarse(pos_tag):
    if pos_tag.startswith('N'):
        return 'Noun'
    elif pos_tag.startswith('V'):
        return 'Verb'
    elif pos_tag.startswith('R'):
        return 'Adverb'
    elif pos_tag.startswith('J'):
        return 'Adjective'
    else:
        return 'Other'

def pos_frequency(text):
    features = []
    pos_tags = pos_tag(text)
    tags = ['Noun', 'Verb', 'Adverb', 'Adjective', 'Other']

    #Weise jedem Wort seine Wortart zu
    pos_tag_frequency = Counter(map_fine_to_coarse(tag) for word, tag in pos_tags)
    # Calculate total number of POS tags
    total_pos_tags = sum(pos_tag_frequency.values())

    # add the relative frequencies as features
    for tag in tags:
      features.append(float(pos_tag_frequency[tag] / total_pos_tags))
    return features

print(pos_frequency(["This", "is", "an", "example", "text", "."]))

**Schritt 5** Wir berechnen unseren Datensatz neu. Das kann etwas dauern, weil POS-Tagging Zeit kostet.

In [ ]:
def initialize_dataset(source):
    all_features = []
    targets = []
    for (sent, label) in source:
        feature_list=[]
        feature_list.append(avg_number_chars(sent))
        feature_list.append(number_words(sent))
        feature_list = feature_list + pos_frequency(sent) # hier ist die entscheidende neue Zeile gegenüber Schritt 3. Alles andere ist gleich.
        all_features.append(feature_list)
        if label=="austen": targets.append(0)
        else: targets.append(1)
    return all_features, targets

train_data, train_targets = initialize_dataset(train_set)
test_data, test_targets = initialize_dataset(test_set)

print (len(train_data), len(train_targets))
print (len(test_data), len(test_targets))

**Schritt 6** Wir lassen den Klassifikator nochmal neu laufen und vergleichen die Ergebnisse.

In [ ]:
text_clf = DecisionTreeClassifier(random_state=42)
text_clf.fit(train_data, train_targets)

# we test and evaluate on the test data
predicted = text_clf.predict(test_data)
evaluate(predicted, test_targets)

# for comparison, we also apply and evaluate on the training data
predicted = text_clf.predict(train_data)
evaluate(predicted, train_targets)

**Aufgabe 3** Vergleichen Sie die Werte. Wurde der ALgorithmus besser?

**Aufgabe 4** Bauen Sie auch die Häufigkeit der häufigsten 20 Wörter als Feature ein. Gehen Sie dazu in 2 Schritten vor:
a) Bestimmen Sie die häufigsten Wörter auf den Trainingsdaten.
b) Bestimmen Sie für jedes dieser Features die relative Häufigkeit für jeden Satz und integrieren Sie das in Ihre Datensatzinitialisierung.